In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import os

stock_directory = 'data/price/raw'
stock_data = {}

csv_files = [file for file in os.listdir(stock_directory) if file.endswith('.csv')]
for file in csv_files:
    symbol = file.split('.')[0]  # Extract stock symbol from filename
    df = pd.read_csv(os.path.join(stock_directory, file))
    df['Date'] = pd.to_datetime(df['Date'])
    df.fillna(method='ffill', inplace=True)
    stock_data[symbol] = df

In [3]:
stock_data['WFC']

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-09-04,34.040001,34.130001,33.730000,33.799999,29.324499,14717600
1,2012-09-05,33.840000,34.009998,33.740002,33.750000,29.281118,16121100
2,2012-09-06,33.980000,34.950001,33.970001,34.840000,30.226799,32143100
3,2012-09-07,34.959999,35.189999,34.740002,35.000000,30.365608,26773900
4,2012-09-10,34.869999,34.939999,34.590000,34.590000,30.009895,19616700
...,...,...,...,...,...,...,...
1253,2017-08-28,51.950001,52.020000,51.419998,51.630001,51.630001,12425900
1254,2017-08-29,51.209999,51.500000,51.130001,51.419998,51.419998,10715500
1255,2017-08-30,51.459999,51.740002,51.200001,51.360001,51.360001,11427500
1256,2017-08-31,51.000000,51.240002,50.869999,51.070000,51.070000,25231100


In [4]:
import os
import json

tweet_directory = 'data/tweet/raw'
tweets_data = {}
tweet_threshold = 500

for stock_folder in os.listdir(tweet_directory):
    stock_path = os.path.join(tweet_directory, stock_folder)
    if os.path.isdir(stock_path):
        all_tweets = []
        count = 0
        for tweet_file in os.listdir(stock_path):
            count += 1
            file_path = os.path.join(stock_path, tweet_file)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        tweet_json = json.loads(line.strip())
                        tweet_data = {
                            'Date': pd.to_datetime(tweet_json['created_at']),
                            'Text': tweet_json['text'],
                            'User': tweet_json['user']['screen_name'],
                            'Followers': tweet_json['user']['followers_count'],
                            'Friends': tweet_json['user']['friends_count']
                        }
                        all_tweets.append(tweet_data)
                    except json.JSONDecodeError:
                        print(f"Error decoding JSON in file {file_path}")
        # if len(all_tweets) >= tweet_threshold:
        if count >= tweet_threshold:
            tweets_data[stock_folder] = pd.DataFrame(all_tweets)

In [5]:
stock_data = {symbol: df for symbol, df in stock_data.items() if symbol in tweets_data}

In [6]:
len(stock_data)

27

In [7]:
for stock, df in tweets_data.items():
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    tweets_data[stock] = df

In [9]:
for stock, df in tweets_data.items():
    grouped = df.groupby('Date').agg({
        'Text': list,          
        'User': list,          
        'Followers': list,     
        'Friends': list        
    }).reset_index()

    tweets_data[stock] = grouped

In [10]:
stock_data['BA']['Date'] = pd.to_datetime(stock_data['BA']['Date'])
tweets_data['BA']['Date'] = pd.to_datetime(tweets_data['BA']['Date'])

common_dates = pd.Series(list(set(stock_data['BA']['Date']).intersection(set(tweets_data['BA']['Date']))))

common_dates_count = len(common_dates)

print("Count of common dates:", common_dates_count)

Count of common dates: 373


In [11]:
for stock in stock_data.keys():
    if stock in tweets_data:
        
        stock_data[stock]['Date'] = pd.to_datetime(stock_data[stock]['Date'])
        tweets_data[stock]['Date'] = pd.to_datetime(tweets_data[stock]['Date'])

        merged_data = stock_data[stock].merge(tweets_data[stock], on='Date', how='left')
        
        stock_data[stock] = merged_data

In [21]:
filtered_data = stock_data['AAPL'][stock_data['AAPL']['Text'].notna()]
filtered_data

,Date,Open,High,Low,Close,Adj Close,Volume,Text,User,Followers,Friends
332,2013-12-31,79.167145,80.182854,79.142860,80.145714,74.571281,55771100,[RT @philstockworld: Summary of Yesterday's We...,[qajpauyl],[6],[105]
333,2014-01-02,79.382858,79.575714,78.860001,79.018570,73.522530,58671200,[$AAPL - Wall St. kicks off New Year on lower ...,"[stock_newsnet, ErasmoWidrick, qxu64gkoyki14, ...","[1369, 21, 5, 834, 141, 225, 1369, 3, 5, 1563,...","[9, 127, 52, 962, 480, 0, 9, 68, 77, 726, 65, ..."
334,2014-01-03,78.980003,79.099998,77.204285,77.282860,71.907555,98116900,[RT @philstockworld: Our Top 3 Trade Ideas for...,"[divinemtgk, FITradingApple, corybolton, Crame...","[4, 268, 614, 224, 688, 197, 3839, 37, 4481, 5...","[66, 332, 710, 0, 1784, 529, 73, 11, 2030, 33,..."
335,2014-01-06,76.778572,78.114288,76.228569,77.704285,72.299644,103152700,[In it to win it: $MU puts working.. $AAPL lo...,"[OnTime_Picks, TheStockHerald, Brandenmart, Wo...","[1298, 9, 217, 250614, 46, 26, 31, 23, 495, 20...","[951, 0, 1122, 39119, 85, 124, 123, 1, 26, 316..."
336,2014-01-07,77.760002,77.994286,76.845711,77.148575,71.782608,79302300,[$AAPL I love my iPad b/c no virus and I am pr...,"[mfd504, iTarunAgarwal, GreaterStocks, Greater...","[60, 196, 3, 3, 3867, 2128, 72, 2668, 439, 138...","[68, 343, 0, 0, 73, 282, 188, 2721, 402, 9, 23..."
...,...,...,...,...,...,...,...,...,...,...,...
893,2016-03-24,105.470001,106.250000,104.889999,105.669998,102.653854,26133000,[https://t.co/Y7XlZ7zeXk Verified $40.43 profi...,"[NexStarMedia, owlerowl, Lastcombo, computer_h...","[1022, 9, 863, 24, 31, 25, 58, 784, 213, 24, 3...","[1923, 0, 387, 2, 194, 2, 161, 858, 319, 3, 2,..."
894,2016-03-28,106.000000,106.190002,105.059998,105.190002,102.187561,19411400,[RT @WSJ: The FBI has accessed San Bernardino ...,"[JuanVato, jmvidalillanes, frosty8556, poznans...","[240, 4834, 1239, 470, 768, 1934, 29, 57, 1369...","[139, 2894, 2734, 3545, 488, 39, 2, 161, 184, ..."
895,2016-03-29,104.889999,107.790001,104.879997,107.680000,104.606491,31190100,[$AAPL Facebook hints at possible Apple Pay te...,"[AppleNewsAAPL, Stockology101, BCAgroup, Ekoor...","[120, 619, 1764, 79, 79, 344, 346, 346, 30, 29...","[57, 649, 1983, 158, 158, 360, 1959, 1959, 2, ..."
896,2016-03-30,108.650002,110.419998,108.599998,109.559998,106.432831,45601100,[Apple Inc. $AAPL Position Reduced by Frontier...,"[thelegacynewswi, TweakTown, MikeTerrell1974, ...","[2135, 22573, 82, 343, 343, 89, 89, 83, 83, 97...","[11, 377, 63, 361, 361, 147, 147, 2, 2, 237, 2..."


In [17]:
filtered_data['Date'].unique()

<DatetimeArray>
['2014-01-02 00:00:00', '2014-01-03 00:00:00', '2014-01-06 00:00:00',
 '2014-01-07 00:00:00', '2014-01-08 00:00:00', '2014-01-09 00:00:00',
 '2014-01-10 00:00:00', '2014-01-14 00:00:00', '2014-01-15 00:00:00',
 '2014-01-16 00:00:00',
 ...
 '2016-03-09 00:00:00', '2016-03-15 00:00:00', '2016-03-16 00:00:00',
 '2016-03-18 00:00:00', '2016-03-21 00:00:00', '2016-03-22 00:00:00',
 '2016-03-23 00:00:00', '2016-03-29 00:00:00', '2016-03-30 00:00:00',
 '2016-03-31 00:00:00']
Length: 373, dtype: datetime64[ns]

In [18]:
stock_data.keys()

dict_keys(['JPM', 'GOOG', 'MCD', 'CSCO', 'BAC', 'JNJ', 'AMZN', 'XOM', 'BA', 'WMT', 'MSFT', 'CELG', 'T', 'BABA', 'PCLN', 'D', 'GE', 'VZ', 'FB', 'KO', 'PFE', 'AAPL', 'V', 'C', 'INTC', 'CVX', 'DIS'])

In [19]:
tweet_data

{'Date': Timestamp('2015-03-16 07:57:28+0000', tz='UTC'),
 'Text': 'Authorities closing in on JPMorgan hackers (JPM) $JPM http://t.co/aGIP36PrA0',
 'User': 'stockwire24',
 'Followers': 1201,
 'Friends': 3}

In [20]:
import pickle

with open('data/stock_data.pkl', 'wb') as file:
    pickle.dump(stock_data, file)